## 1-Safety- crash frequency

In [6]:
import geopandas as gpd
import pandas as pd
import folium
import json
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
# geojson_data = {
#     "type": "FeatureCollection",
#     "features": [
#         {
#             "type": "Feature",
#             "properties": {
#                 "project_id": 1,
#                 "type": "Highway",
#                 "syip": "No",
#                 "county": "Hopewell, VA",
#                 "cmf": 0.25,
#                 "AADT": 30601,
#                 "length": 0.90,
#                 "fc": "PA",
#                 "cost_mil": 14.3
#             },
#             "geometry": {"type": "Point", "coordinates": [-77.29355, 37.30849]}
#         },
#         {
#             "type": "Feature",
#             "properties": {
#                 "project_id": 2,
#                 "type": "Intersection",
#                 "syip": "Yes",
#                 "county": "Hopewell, VA",
#                 "cmf": 0.37,
#                 "AADT": 15407,
#                 "length": 0.40,
#                 "fc": "MA",
#                 "cost_mil": 5.8
#             },
#             "geometry": {"type": "Point", "coordinates": [-77.290, 37.290]}
#         },
#         {
#             "type": "Feature",
#             "properties": {
#                 "project_id": 3,
#                 "type": "Intersection",
#                 "syip": "Yes",
#                 "county": "Hopewell, VA",
#                 "cmf": 0.48,
#                 "AADT": 7357,
#                 "length": 0.80,
#                 "fc": "MC",
#                 "cost_mil": 6.7
#             },
#             "geometry": {"type": "Point", "coordinates": [-77.289, 37.302]}
#         }
#     ]
# }

In [ ]:
gdf = gpd.read_file(r'C:\Users\Adamu\Documents\client - Copy\stbg_projects_test\projects.geojson')

In [ ]:
crashes = gpd.read_file(r'C:\Users\Adamu\Documents\client - Copy\stbg_projects_test\crashes.geojson')

In [ ]:
# --------------------
# Load data
# --------------------
# gdf = gpd.read_file("gdf.geojson")
# crashes = gpd.read_file("crashes.geojson")

# --------------------
# Ensure projected CRS for buffer
# --------------------
gdf = gdf.to_crs(epsg=2263)       # feet
crashes = crashes.to_crs(gdf.crs)

# --------------------
# Add project_id if not present
# --------------------
if "project_id" not in gdf.columns:
    gdf["project_id"] = range(1, len(gdf) + 1)

# --------------------
# Step 1: Create 250 ft buffer
# --------------------
gdf_buffered = gdf.copy()
gdf_buffered["geometry"] = gdf_buffered.geometry.buffer(250)

# --------------------
# Step 2: Select crashes that intersect buffer
# --------------------
crashes_in_buffer = gpd.sjoin(
    crashes,
    gdf_buffered[["project_id", "geometry"]],
    how="inner",
    predicate="intersects"
)

# --------------------
# Step 3: Summarize crash people counts per buffer (group by project_id)
# --------------------
crash_sums = crashes_in_buffer.groupby("project_id").agg({
    "K_PEOPLE": "sum",
    "A_PEOPLE": "sum",
    "B_PEOPLE": "sum",
    "C_PEOPLE": "sum"
}).reset_index()

# Merge summary back to buffers
gdf_buffered = gdf_buffered.merge(crash_sums, on="project_id", how="left")
gdf_buffered.fillna(0, inplace=True)

# --------------------
# Reproject back to WGS84 for saving/visualization
# --------------------
gdf_buffered = gdf_buffered.to_crs(epsg=4326)

# 4. Calculate EPDO
gdf_buffered["EPDO"] = (
    gdf_buffered["K_PEOPLE"] * 2715000 +
    gdf_buffered["A_PEOPLE"] * 2715000 +
    gdf_buffered["B_PEOPLE"] * 300000 +
    gdf_buffered["C_PEOPLE"] * 170000
)

# 5. Calculate benefit = EPDO * (1 - cmf)
# Assuming your projects file has a 'cmf' column
gdf_buffered["benefit"] = gdf_buffered["EPDO"] * (1 - gdf_buffered["cmf"])
# --------------------
# 6: Calculate safety score
# --------------------
max_benefit = gdf_buffered["benefit"].max()

# Avoid divide by zero
if max_benefit > 0:
    gdf_buffered["safety_freq"] = (gdf_buffered["benefit"] / max_benefit) * 50
else:
    gdf_buffered["safety_freq"] = 0

safety_freq = gdf_buffered
safety_freq = safety_freq[['project_id','safety_freq']]

In [ ]:
safety_freq

,project_id,safety_freq
0,1,50.000000
1,2,0.000000
2,3,3.189896


## 2-Safety- crash rate

In [ ]:
# aadt shd be in project file. if intersection project,  
# the annual traffic volume is calculated
# as the weighted average volume for all segments.

# aadt_values = [30601, 15407, 7357] 
# gdf_buffered["AADT"] = aadt_values

# length_values = [0.90, 0.40, 0.80]
# gdf_buffered["length"] = length_values

# Define epdo_rate based on project type
# Function to calculate epdo_rate and store denominator (VMT)
def calculate_epdo_rate_and_vmt(row):
    if row["type"].lower() == "highway":
        vmt = row["AADT"] * row["length"] * 365 / 100_000_000
    elif row["type"].lower() == "intersection":
        vmt = row["AADT"] * 365 / 1_000_000
    else:
        vmt = 1  # avoid division by zero
    epdo_rate = row["benefit"] / vmt if vmt != 0 else 0
    return pd.Series({"VMT": vmt, "epdo_rate": epdo_rate})

# Apply function
gdf_buffered[["VMT", "epdo_rate"]] = gdf_buffered.apply(calculate_epdo_rate_and_vmt, axis=1)

# Calculate safety_rate
max_rate = gdf_buffered["epdo_rate"].max()

# Avoid division by zero
if max_rate > 0:
    gdf_buffered["safety_rate"] = (gdf_buffered["epdo_rate"] / max_rate) * 50
else:
    gdf_buffered["safety_rate"] = 0

safety_rate = gdf_buffered[['project_id','safety_rate']]

## 3-congestion-demand

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

# Load projects
projects = gpd.read_file(r'C:\Users\Adamu\Documents\client - Copy\stbg_projects_test\projects.geojson')

# Load AADT segments
aadt = gpd.read_file(r'C:\Users\Adamu\Documents\client - Copy\stbg_projects_test\stbg_aadt.geojson')

# Ensure both are in the same CRS (projected CRS for distance calculations, e.g., meters)
projects = projects.to_crs(epsg=2283)  # Virginia State Plane, or use suitable CRS
aadt = aadt.to_crs(projects.crs)

buffer_distance = 0.25 * 1609.34  # meters
projects["buffer"] = projects.geometry.buffer(buffer_distance)

# Convert project buffers to GeoDataFrame
project_buffers = projects.set_geometry("buffer")

# Perform spatial join
intersected = gpd.sjoin(aadt, project_buffers, how="inner", predicate="intersects")

intersected["segment_mileage"] = intersected.geometry.length / 1609.34  # meters → miles
intersected["vmt"] = intersected["aadt_0"] * intersected["segment_mileage"]

wa_aadt = (
    intersected.groupby("project_id")
    .apply(lambda x: x["vmt"].sum() / x["segment_mileage"].sum())
    .reset_index(name="wa_aadt")
)

# Drop any existing wa_aadt columns to avoid _x/_y
projects = projects.drop(columns=[col for col in projects.columns if "wa_aadt" in col], errors="ignore")

# Merge the computed wa_aadt
projects = projects.merge(wa_aadt, on="project_id", how="left")

# Replace NaN with 0
projects["wa_aadt"] = projects["wa_aadt"].fillna(0)

# normalize
projects["cong_demand"] = (projects["wa_aadt"] / projects["wa_aadt"].max()) * 10
projects = projects[['project_id','cong_demand']]
cong_demand = projects

C:\Users\zmumuni\AppData\Local\Temp\ipykernel_26232\507337334.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x["vmt"].sum() / x["segment_mileage"].sum())


In [ ]:
cong_demand

,project_id,cong_demand
0,1,10.00000
1,2,0.00000
2,3,3.37701


## 4-congestion-los

In [ ]:
aadt.head()

,RTE_NM,aadt_0,los_0,FROM_MEASU,TO_MEASURE,len,geometry
0,S-VA123PR RIVES RD,6918,A,0.27,0.46,0.19,"MULTILINESTRING ((11816309.824 6384822.946, 11..."
1,R-VA020SC01106WB,1809,A,0.69,0.79,0.10,"MULTILINESTRING ((11796177.94 6411032.086, 117..."
2,R-VA020SC07601EB,0,None,0.00,0.04,0.04,"MULTILINESTRING ((11767053.551 6437754.187, 11..."
3,S-VA123PR SOUTH BLVD,2819,A,1.11,1.17,0.06,"MULTILINESTRING ((11808338.289 6395667.842, 11..."
4,R-VA116UR09043NB,4727,A,1.46,1.48,0.02,"MULTILINESTRING ((11833513.157 6424432.641, 11..."


In [ ]:
# 1. Load your project and AADT layers
projects = gpd.read_file(r'C:\Users\Adamu\Documents\client - Copy\stbg_projects_test\projects.geojson')
aadt = aadt          # replace with your file

# 2. Create cong_value column based on los_0
los_mapping = {
    "A": 0,
    "B": 1,
    "C": 2,
    "D": 3,
    "E": 3,
    "F": 3
}

aadt["cong_value"] = aadt["los_0"].map(los_mapping)

# 3. Create 0.25 mile buffer around project locations
# 0.25 miles ≈ 402.336 meters (assuming CRS in meters)
projects = projects.to_crs(epsg=3857)  # project to metric CRS
aadt = aadt.to_crs(epsg=3857)

projects["buffer"] = projects.geometry.buffer(402.336)

# 4. Intersect buffer and AADT segments
# Explode buffers if needed
projects_exploded = projects.explode(index_parts=False)
intersected = gpd.overlay(aadt, gpd.GeoDataFrame(geometry=projects_exploded["buffer"]), how="intersection")

# 5. Sum cong_value for all segments in each project buffer
# Assuming you have a project ID column in projects, say 'project_id'
intersected = intersected.merge(projects[["project_id", "buffer"]], left_on='geometry', right_on='buffer', how='left')

project_cong = (
    intersected.groupby("project_id")["cong_value"]
    .sum()
    .reset_index(name="sum_cong_value")
)

# Merge back to projects if desired
projects = projects.merge(project_cong, on="project_id", how="left")

projects["sum_cong_value"] = projects["sum_cong_value"].fillna(0)

# normalize 
normalized = (projects["sum_cong_value"] / projects["sum_cong_value"].max()) * 5
# If indivisible (not integer), return 0
projects["cong_los"] = normalized.where(normalized % 1 == 0, 0)

projects = projects[['project_id','cong_los']]
cong_los = projects

In [ ]:
cong_los

,project_id,cong_los
0,1,0.0
1,2,0.0
2,3,0.0


## 5-equity/access-access to jobs

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

# --- Load your datasets ---
# TAZ dataframe with geometry
pop_emp_df = gpd.read_file(r"C:\Users\Adamu\Documents\client - Copy\stbg_projects_test\pop_emp_df.geojson")  # replace with actual file

# Projects from geojson dict
projects = gpd.read_file(r'C:\Users\Adamu\Documents\client - Copy\stbg_projects_test\projects.geojson')

# --- Define buffer distances in meters (GeoPandas requires projected CRS) ---
fc_distances_miles = {"PA": 10, "MA": 7.5, "MC": 5}
mile_to_meter = 1609.34
fc_distances_m = {k: v * mile_to_meter for k, v in fc_distances_miles.items()}

# Project both datasets to a projected CRS for buffering (Virginia = EPSG:2283 or use EPSG:3857)
projects = projects.to_crs(epsg=2283)
pop_emp_df = pop_emp_df.to_crs(epsg=2283)

# --- Calculate TAZ centroids ---
pop_emp_df["centroid"] = pop_emp_df.geometry.centroid

results = []

# --- For each project ---
for _, proj in projects.iterrows():
    fc = proj["fc"]
    buffer_dist = fc_distances_m[fc]
    
    # Create buffer
    proj_buffer = proj.geometry.buffer(buffer_dist)
    
    # Select TAZs with centroid inside buffer
    selected = pop_emp_df[pop_emp_df["centroid"].within(proj_buffer)]
    
    # Aggregate employment
    sum_emp17 = selected["emp17"].sum()
    sum_emp50 = selected["emp50"].sum()
    
    # % change
    pct_change = ((sum_emp50 - sum_emp17) / sum_emp17 * 100) if sum_emp17 != 0 else 0
    
    results.append({
        "project_id": proj["project_id"],
        "sum_emp17": sum_emp17,
        "sum_emp50": sum_emp50,
        "pct_change": pct_change
    })

# Convert to DataFrame
results_df = pd.DataFrame(results)

results_df["jobs_pc"] = (results_df["pct_change"] / results_df["pct_change"].max()) * 5 if results_df["pct_change"].max() != 0 else 0

results_df = results_df[['project_id','jobs_pc']]
eq_acc_jobs = results_df

NameError: name 'gdf' is not defined

In [ ]:
eq_acc_jobs

,project_id,jobs_pc
0,1,5.000000
1,2,0.283142
2,3,0.289369


## 6-equity/access-access to jobs -ej

In [ ]:
import geopandas as gpd
import pandas as pd

# --- Load datasets ---
pop_emp_df = gpd.read_file(r"C:\Users\Adamu\Documents\client - Copy\stbg_projects_test\pop_emp_df.geojson")   # TAZ layer with emp17/emp50
ej = gpd.read_file(r"C:\Users\Adamu\Documents\client - Copy\stbg_projects_test\t6.geojson")            # EJ polygons
projects = gpd.read_file(r'C:\Users\Adamu\Documents\client - Copy\stbg_projects_test\projects.geojson')

# --- Distances in miles → meters ---
fc_distances_miles = {"PA": 10, "MA": 7.5, "MC": 5}
mile_to_meter = 1609.34
fc_distances_m = {k: v * mile_to_meter for k, v in fc_distances_miles.items()}

# --- Project all layers to a projected CRS for spatial analysis ---
projects = projects.to_crs(epsg=2283)
pop_emp_df = pop_emp_df.to_crs(epsg=2283)
ej = ej.to_crs(epsg=2283)

results = []

# --- Loop through projects ---
for _, proj in projects.iterrows():
    fc = proj["fc"]
    buffer_dist = fc_distances_m[fc]
    
    # Create buffer
    proj_buffer = proj.geometry.buffer(buffer_dist)
    
    # Clip EJ polygons within buffer
    ej_clip = ej[ej.intersects(proj_buffer)]
    
    if ej_clip.empty:
        sum_emp17 = 0
        sum_emp50 = 0
    else:
        # Clip TAZ by EJ polygons (intersection)
        taz_ej_intersect = gpd.overlay(pop_emp_df, ej_clip, how="intersection")
        
        # Aggregate employment in intersected areas
        sum_emp17 = taz_ej_intersect["emp17"].sum()
        sum_emp50 = taz_ej_intersect["emp50"].sum()
    
    # % change
    pct_change = ((sum_emp50 - sum_emp17) / sum_emp17 * 100) if sum_emp17 != 0 else 0
    
    results.append({
        "project_id": proj["project_id"],
        "sum_emp17": sum_emp17,
        "sum_emp50": sum_emp50,
        "pct_change": pct_change
    })

# --- Results DataFrame ---
results_df = pd.DataFrame(results)

# --- Normalize percent change (one-liner) ---
results_df["jobs_pc_ej"] = (results_df["pct_change"] / results_df["pct_change"].max()) * 5 if results_df["pct_change"].max() != 0 else 0
results_df = results_df[['project_id','jobs_pc_ej']]

eq_acc_jobs_ej = results_df

In [ ]:
eq_acc_jobs_ej

,project_id,jobs_pc_ej
0,1,4.497950
1,2,5.000000
2,3,4.666751


## 7-access to Non-work dest

In [ ]:
# groc = gpd.read_file(r"C:\Users\zmumuni\stbg_projects_test\non-work destinations for mapping\Groceries for mapping.shp")                   # NW polygons
# med = gpd.read_file(r"C:\Users\zmumuni\stbg_projects_test\non-work destinations for mapping\medical for mapping.shp")                   # NW polygons
# park = gpd.read_file(r"C:\Users\zmumuni\stbg_projects_test\non-work destinations for mapping\Parks for mapping.shp")                   # NW polygons
# sch = gpd.read_file(r"C:\Users\zmumuni\stbg_projects_test\non-work destinations for mapping\schools libraries government for mapping.shp")                   # NW polygons

# # Reproject if necessary (example: reproject all to shp1's CRS)
# med = med.to_crs(groc.crs)
# park = park.to_crs(groc.crs)
# sch = sch.to_crs(groc.crs)

# # Combine shapefiles
# combined = gpd.GeoDataFrame(pd.concat([groc, med, park, sch], ignore_index=True))

# combined.to_file(r"C:\Users\zmumuni\stbg_projects_test\nw.geojson", driver="GeoJSON")

In [ ]:
import geopandas as gpd
import pandas as pd

# --- Load datasets ---
pop_emp_df = gpd.read_file(r"C:\Users\Adamu\Documents\client - Copy\stbg_projects_test\pop_emp_df.geojson")  # TAZ layer
nw = gpd.read_file(r"C:\Users\Adamu\Documents\client - Copy\stbg_projects_test\nw.geojson")                  # NW points
projects = gpd.read_file(r'C:\Users\Adamu\Documents\client - Copy\stbg_projects_test\projects.geojson')

# --- Distances (miles -> meters) ---
fc_distances_miles = {"PA": 10, "MA": 7.5, "MC": 5}
mile_to_meter = 1609.34
fc_distances_m = {k: v * mile_to_meter for k, v in fc_distances_miles.items()}

# --- Project to planar CRS ---
projects = projects.to_crs(epsg=2283)
nw = nw.to_crs(epsg=2283)
pop_emp_df = pop_emp_df.to_crs(epsg=2283)

results = []

for _, proj in projects.iterrows():
    fc = proj["fc"]
    buffer_dist = fc_distances_m[fc]
    
    # Buffer project
    proj_buffer = proj.geometry.buffer(buffer_dist)
    
    # Count NW points inside buffer
    nw_count = nw[nw.within(proj_buffer)].shape[0]
    
    # Intersect TAZs with buffer
    taz_selected = pop_emp_df[pop_emp_df.intersects(proj_buffer)]
    
    if taz_selected.empty:
        sum_emp2017 = sum_emp2050 = sum_pop2017 = sum_pop2050 = area_sqmi = 0
    else:
        sum_emp2017 = taz_selected["emp17"].sum()
        sum_emp2050 = taz_selected["emp50"].sum()
        sum_pop2017 = taz_selected["pop17"].sum()
        sum_pop2050 = taz_selected["pop50"].sum()
        
        # area in square miles
        area_sqmi = taz_selected.to_crs(epsg=3857).geometry.area.sum() / (1609.34**2)
    
    # Calculate density metrics
    if area_sqmi > 0:
        pop_emp_den_2017 = nw_count * (sum_emp2017 + sum_pop2017) / area_sqmi
        pop_emp_den_2050 = nw_count * (sum_emp2050 + sum_pop2050) / area_sqmi
    else:
        pop_emp_den_2017 = pop_emp_den_2050 = 0
    
    results.append({
        "project_id": proj["project_id"],
        "nw_count": nw_count,
        "sum_emp2017": sum_emp2017,
        "sum_emp2050": sum_emp2050,
        "sum_pop2017": sum_pop2017,
        "sum_pop2050": sum_pop2050,
        "area_sqmi": area_sqmi,
        "pop_emp_den_2017": pop_emp_den_2017,
        "pop_emp_den_2050": pop_emp_den_2050
    })

# --- Results dataframe ---
results_df = pd.DataFrame(results)

# Percent change in pop_emp_den
results_df["access_nw_pct"] = (
    ((results_df["pop_emp_den_2050"] - results_df["pop_emp_den_2017"]) / results_df["pop_emp_den_2017"] * 100)
    .fillna(0)  # handle division by zero
)

# Normalize percent change to 0–5 scale
results_df["access_nw_norm"] = (
    (results_df["access_nw_pct"] / results_df["access_nw_pct"].max() * 5) 
    if results_df["access_nw_pct"].max() != 0 else 0
)

results_df = results_df[['project_id','access_nw_norm']]
eq_acc_nw = results_df

In [ ]:
eq_acc_nw

,project_id,access_nw_norm
0,1,5.000000
1,2,3.889575
2,3,2.886672


## 8-access to Non-work dest - ej

In [ ]:
import geopandas as gpd
import pandas as pd

# --- Load datasets ---
pop_emp_df = gpd.read_file(r"C:\Users\Adamu\Documents\client - Copy\stbg_projects_test\pop_emp_df.geojson")  # TAZ layer
nw = gpd.read_file(r"C:\Users\Adamu\Documents\client - Copy\stbg_projects_test\nw.geojson")                  # NW points
t6 = gpd.read_file(r"C:\Users\Adamu\Documents\client - Copy\stbg_projects_test\t6.geojson")                  # T6 polygons
projects = gpd.read_file(r'C:\Users\Adamu\Documents\client - Copy\stbg_projects_test\projects.geojson')

# --- Distances (miles -> meters) ---
fc_distances_miles = {"PA": 10, "MA": 7.5, "MC": 5}
mile_to_meter = 1609.34
fc_distances_m = {k: v * mile_to_meter for k, v in fc_distances_miles.items()}

# --- Project to planar CRS ---
projects = projects.to_crs(epsg=2283)
nw = nw.to_crs(epsg=2283)
pop_emp_df = pop_emp_df.to_crs(epsg=2283)
t6 = t6.to_crs(epsg=2283)

results = []

for _, proj in projects.iterrows():
    fc = proj["fc"]
    buffer_dist = fc_distances_m[fc]
    
    # Buffer project
    proj_buffer = proj.geometry.buffer(buffer_dist)
    
    # Count NW points inside buffer AND inside T6 polygon
    nw_count = nw[nw.within(proj_buffer) & nw.within(t6.unary_union)].shape[0]
    
    # Intersect TAZs with buffer AND T6 polygon
    taz_selected = pop_emp_df[pop_emp_df.intersects(proj_buffer) & pop_emp_df.intersects(t6.unary_union)]
    
    if taz_selected.empty:
        sum_emp2017 = sum_emp2050 = sum_pop2017 = sum_pop2050 = area_sqmi = 0
    else:
        sum_emp2017 = taz_selected["emp17"].sum()
        sum_emp2050 = taz_selected["emp50"].sum()
        sum_pop2017 = taz_selected["pop17"].sum()
        sum_pop2050 = taz_selected["pop50"].sum()
        
        # area in square miles
        area_sqmi = taz_selected.to_crs(epsg=3857).geometry.area.sum() / (1609.34**2)
    
    # Calculate density metrics
    if area_sqmi > 0:
        pop_emp_den_2017 = nw_count * (sum_emp2017 + sum_pop2017) / area_sqmi
        pop_emp_den_2050 = nw_count * (sum_emp2050 + sum_pop2050) / area_sqmi
    else:
        pop_emp_den_2017 = pop_emp_den_2050 = 0
    
    results.append({
        "project_id": proj["project_id"],
        "nw_count": nw_count,
        "sum_emp2017": sum_emp2017,
        "sum_emp2050": sum_emp2050,
        "sum_pop2017": sum_pop2017,
        "sum_pop2050": sum_pop2050,
        "area_sqmi": area_sqmi,
        "pop_emp_den_2017": pop_emp_den_2017,
        "pop_emp_den_2050": pop_emp_den_2050
    })

# --- Results dataframe ---
results_df = pd.DataFrame(results)

# Percent change in pop_emp_den
results_df["access_nw_pct"] = (
    ((results_df["pop_emp_den_2050"] - results_df["pop_emp_den_2017"]) / results_df["pop_emp_den_2017"] * 100)
    .fillna(0)  # handle division by zero
)

# Normalize percent change to 0–5 scale
max_pct = results_df["access_nw_pct"].max()
results_df["access_nw_ej_norm"] = (results_df["access_nw_pct"] / max_pct * 5) if max_pct != 0 else 0

results_df = results_df[['project_id','access_nw_ej_norm']]
eq_acc_nw_ej = results_df

C:\Users\zmumuni\AppData\Local\Temp\ipykernel_26232\3958568846.py:31: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  nw_count = nw[nw.within(proj_buffer) & nw.within(t6.unary_union)].shape[0]
C:\Users\zmumuni\AppData\Local\Temp\ipykernel_26232\3958568846.py:34: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  taz_selected = pop_emp_df[pop_emp_df.intersects(proj_buffer) & pop_emp_df.intersects(t6.unary_union)]
C:\Users\zmumuni\AppData\Local\Temp\ipykernel_26232\3958568846.py:31: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  nw_count = nw[nw.within(proj_buffer) & nw.within(t6.unary_union)].shape[0]
C:\Users\zmumuni\AppData\Local\Temp\ipykernel_26232\3958568846.py:34: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  taz_selected = pop_emp_df[pop_emp_df.intersects(proj_buf

In [ ]:
eq_acc_nw_ej

,project_id,access_nw_ej_norm
0,1,4.677155
1,2,5.000000
2,3,3.464419


## scoring

In [ ]:
import pandas as pd
from functools import reduce

# Assuming 'gdf' is your GeoDataFrame with the geometry and cost_mil
# List of all other dataframes
dfs = [
    safety_freq,
    safety_rate,
    cong_demand,
    cong_los,
    eq_acc_jobs,
    eq_acc_jobs_ej,
    eq_acc_nw,
    eq_acc_nw_ej
]

# 1. First, merge all the regular dataframes on 'project_id'
merged_data_df = reduce(lambda left, right: pd.merge(left, right, on="project_id", how="outer"), dfs)

# 2. Extract the non-geometry data from the GeoDataFrame to merge it.
# We create a temporary DataFrame from gdf without the geometry column.
gdf_data = gdf

# 3. Merge the combined data with the GeoDataFrame's attribute data
final_attributes_df = pd.merge(merged_data_df, gdf_data, on="project_id", how="outer")

# 4. Now, join this final attribute table back to the original GeoDataFrame to get the geometry.
# This ensures we keep only the projects that are in our original gdf and preserves the geometry.
final_gdf = gdf[['project_id', 'geometry']].merge(final_attributes_df, on='project_id', how='right')

# final_gdf is now a complete GeoDataFrame with all merged data and geometry.

In [ ]:
final_gdf

,project_id,geometry_x,safety_freq,safety_rate,cong_demand,cong_los,jobs_pc,jobs_pc_ej,access_nw_norm,access_nw_ej_norm,type,syip,county,cmf,AADT,length,fc,cost_mil,geometry_y
0,1,POINT (24763.079 -1023725.166),50.000000,50.000000,10.00000,0.0,5.000000,4.497950,5.000000,4.677155,Highway,No,"Hopewell, VA",0.25,30601,0.9,PA,14.3,POINT (24763.079 -1023725.166)
1,2,POINT (25543.512 -1030504.446),0.000000,0.000000,0.00000,0.0,0.283142,5.000000,3.889575,5.000000,Intersection,Yes,"Hopewell, VA",0.37,15407,0.4,MA,5.8,POINT (25543.512 -1030504.446)
2,3,POINT (25999.103 -1026140.848),3.189896,0.119414,3.37701,0.0,0.289369,4.666751,2.886672,3.464419,Intersection,Yes,"Hopewell, VA",0.48,7357,0.8,MC,6.7,POINT (25999.103 -1026140.848)


In [ ]:
# # 1. Calculate the total benefit score
# # Assuming these are the column names in your final_attributes_df or final_gdf
# benefit_columns = [
#     'safety_freq', 
#     'safety_rate', 
#     'cong_demand', 
#     'cong_los', 
#     'jobs_pc', 
#     'jobs_pc_ej', 
#     'access_nw_norm', 
#     'access_nw_ej_norm'
# ]

# # Sum the selected columns to create a new 'benefit' column
# final_gdf['benefit'] = final_gdf[benefit_columns].sum(axis=1)

# # 2. Calculate the Benefit-Cost Ratio (BCR)
# final_gdf['bcr'] = final_gdf['benefit'] / final_gdf['cost_mil']

# # 3. Rank the projects based on the BCR (higher BCR is better)
# final_gdf['rank'] = final_gdf['bcr'].rank(ascending=False, method='dense').astype(int)

# # Show the results sorted by the best BCR
# rank = final_gdf[['project_id', 'type', 'county', 'benefit', 'cost_mil', 'bcr', 'rank']]

In [ ]:
# 1. First convert geometry to geometry_x and geometry_y
final_gdf['geometry_x'] = final_gdf.geometry.x
final_gdf['geometry_y'] = final_gdf.geometry.y

# 2. Calculate the total benefit score
benefit_columns = [
    'safety_freq', 
    'safety_rate', 
    'cong_demand', 
    'cong_los', 
    'jobs_pc', 
    'jobs_pc_ej', 
    'access_nw_norm', 
    'access_nw_ej_norm'
]

# Sum the selected columns to create a new 'benefit' column
final_gdf['benefit'] = final_gdf[benefit_columns].sum(axis=1)

# 3. Calculate the Benefit-Cost Ratio (BCR)
final_gdf['bcr'] = final_gdf['benefit'] / final_gdf['cost_mil']

# 4. Rank the projects based on the BCR (higher BCR is better)
final_gdf['rank'] = final_gdf['bcr'].rank(ascending=False, method='dense').astype(int)

# 5. Create the final table with all columns including geometry coordinates
columns = [
    'project_id',
    'geometry_x',
    'geometry_y',
    'safety_freq',
    'safety_rate',
    'cong_demand',
    'cong_los',
    'jobs_pc',
    'jobs_pc_ej',
    'access_nw_norm',
    'access_nw_ej_norm',
    'type',
    'county',
    'benefit',
    'cost_mil',
    'bcr',
    'rank'
]

# Format all numeric columns to 6 decimal places
numeric_columns = [
    'geometry_x', 'geometry_y',
    'safety_freq', 'safety_rate',
    'cong_demand', 'cong_los',
    'jobs_pc', 'jobs_pc_ej',
    'access_nw_norm', 'access_nw_ej_norm',
    'benefit', 'cost_mil', 'bcr'
]

# Create the rank DataFrame with formatted numbers
rank = final_gdf[columns].copy()
for col in numeric_columns:
    rank[col] = rank[col].apply(lambda x: f"{x:.6f}")

# Sort by rank
rank = rank.sort_values('rank')

In [53]:
rank

,project_id,safety_freq,safety_rate,cong_demand,cong_los,jobs_pc,jobs_pc_ej,access_nw_norm,access_nw_ej_norm,type,county,benefit,cost_mil,bcr,rank
0,1,50.000000,50.000000,10.00000,0.0,5.000000,4.497950,5.000000,4.677155,Highway,"Hopewell, VA",129.175105,14.3,9.033224,1
2,3,3.189896,0.119414,3.37701,0.0,0.289369,4.666751,2.886672,3.464419,Intersection,"Hopewell, VA",17.993531,6.7,2.685602,2
1,2,0.000000,0.000000,0.00000,0.0,0.283142,5.000000,3.889575,5.000000,Intersection,"Hopewell, VA",14.172718,5.8,2.443572,3


## 9- Sensitive Features